In [ ]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

"""
After auditing is complete the next step is to prepare the data to be inserted into a SQL database.
To do so you will parse the elements in the OSM XML file, transforming them from document format to
tabular format, thus making it possible to write to .csv files.  These csv files can then easily be
imported to a SQL database as tables.

The process for this transformation is as follows:
- Use iterparse to iteratively step through each top level element in the XML
- Shape each element into several data structures using a custom function
- Utilize a schema and validation library to ensure the transformed data is in the correct format
- Write each data structure to the appropriate .csv files

We've already provided the code needed to load the data, perform iterative parsing and write the
output to csv files. Your task is to complete the shape_element function that will transform each
element into the correct format. To make this process easier we've already defined a schema (see
the schema.py file in the last code tab) for the .csv files and the eventual tables. Using the 
cerberus library we can validate the output against this schema to ensure it is correct.

## Shape Element Function
The function should take as input an iterparse Element object and return a dictionary.

### If the element top level tag is "node":
The dictionary returned should have the format {"node": .., "node_tags": ...}

The "node" field should hold a dictionary of the following top level node attributes:
- id
- user
- uid
- version
- lat
- lon
- timestamp
- changeset
All other attributes can be ignored

The "node_tags" field should hold a list of dictionaries, one per secondary tag. Secondary tags are
child tags of node which have the tag name/type: "tag". Each dictionary should have the following
fields from the secondary tag attributes:
- id: the top level node id attribute value
- key: the full tag "k" attribute value if no colon is present or the characters after the colon if one is.
- value: the tag "v" attribute value
- type: either the characters before the colon in the tag "k" value or "regular" if a colon
        is not present.

Additionally,

- if the tag "k" value contains problematic characters, the tag should be ignored
- if the tag "k" value contains a ":" the characters before the ":" should be set as the tag type
  and characters after the ":" should be set as the tag key
- if there are additional ":" in the "k" value they and they should be ignored and kept as part of
  the tag key. For example:

  <tag k="addr:street:name" v="Lincoln"/>
  should be turned into
  {'id': 12345, 'key': 'street:name', 'value': 'Lincoln', 'type': 'addr'}

- If a node has no secondary tags then the "node_tags" field should just contain an empty list.

The final return value for a "node" element should look something like:

{'node': {'id': 757860928,
          'user': 'uboot',
          'uid': 26299,
       'version': '2',
          'lat': 41.9747374,
          'lon': -87.6920102,
          'timestamp': '2010-07-22T16:16:51Z',
      'changeset': 5288876},
 'node_tags': [{'id': 757860928,
                'key': 'amenity',
                'value': 'fast_food',
                'type': 'regular'},
               {'id': 757860928,
                'key': 'cuisine',
                'value': 'sausage',
                'type': 'regular'},
               {'id': 757860928,
                'key': 'name',
                'value': "Shelly's Tasty Freeze",
                'type': 'regular'}]}

### If the element top level tag is "way":
The dictionary should have the format {"way": ..., "way_tags": ..., "way_nodes": ...}

The "way" field should hold a dictionary of the following top level way attributes:
- id
-  user
- uid
- version
- timestamp
- changeset

All other attributes can be ignored

The "way_tags" field should again hold a list of dictionaries, following the exact same rules as
for "node_tags".

Additionally, the dictionary should have a field "way_nodes". "way_nodes" should hold a list of
dictionaries, one for each nd child tag.  Each dictionary should have the fields:
- id: the top level element (way) id
- node_id: the ref attribute value of the nd tag
- position: the index starting at 0 of the nd tag i.e. what order the nd tag appears within
            the way element

The final return value for a "way" element should look something like:

{'way': {'id': 209809850,
         'user': 'chicago-buildings',
         'uid': 674454,
         'version': '1',
         'timestamp': '2013-03-13T15:58:04Z',
         'changeset': 15353317},
 'way_nodes': [{'id': 209809850, 'node_id': 2199822281, 'position': 0},
               {'id': 209809850, 'node_id': 2199822390, 'position': 1},
               {'id': 209809850, 'node_id': 2199822392, 'position': 2},
               {'id': 209809850, 'node_id': 2199822369, 'position': 3},
               {'id': 209809850, 'node_id': 2199822370, 'position': 4},
               {'id': 209809850, 'node_id': 2199822284, 'position': 5},
               {'id': 209809850, 'node_id': 2199822281, 'position': 6}],
 'way_tags': [{'id': 209809850,
               'key': 'housenumber',
               'type': 'addr',
               'value': '1412'},
              {'id': 209809850,
               'key': 'street',
               'type': 'addr',
               'value': 'West Lexington St.'},
              {'id': 209809850,
               'key': 'street:name',
               'type': 'addr',
               'value': 'Lexington'},
              {'id': '209809850',
               'key': 'street:prefix',
               'type': 'addr',
               'value': 'West'},
              {'id': 209809850,
               'key': 'street:type',
               'type': 'addr',
               'value': 'Street'},
              {'id': 209809850,
               'key': 'building',
               'type': 'regular',
               'value': 'yes'},
              {'id': 209809850,
               'key': 'levels',
               'type': 'building',
               'value': '1'},
              {'id': 209809850,
               'key': 'building_id',
               'type': 'chicago',
               'value': '366409'}]}
"""

In [8]:
import csv
import codecs
import pprint
import re
import xml.etree.cElementTree as ET

import cerberus

import schema

In [9]:
def insert_spaces(number):
    '''Insert spaces acc to the format'''
    number = number[:3] + ' ' + number[3:5] + ' ' + number[5:]
    return number
    
def get_raw_local(number):
    
    '''Clean the number and get raw local num'''
        #Delete special characters an spaces
    char = [' ', '(', ')', '-', '/',';', ',', '_']  
    number = filter(lambda x: not (x in char), number)

    #get the local number
    number = number.lstrip('+')
    number = number.lstrip('00')
    number = number.lstrip('48')

    # ditch the second number or extra digits over 9
    if len(number)>=9:
        number = number[:9]

    return number
        
def is_foreign(number):
    '''Check if foreign number'''
    
    #Delete special characters an spaces
    char = [' ', '(', ')', '-', '/',';']  
    number = filter(lambda x: not (x in char), number)
    
    #Check if foreign
    if number.startswith('00') or number.startswith('+'):
        number = number.lstrip('+')
        number = number.lstrip('00')
        if not (number.startswith('48') or number.startswith('22')):
            return True
 
def correct_phone(number):
    if not is_foreign(number):
        number =  get_raw_local(number)

        #add the country prefix
        number = ('+48'+ number)

    number = insert_spaces(number)   
    return number
        

In [10]:
def correct_building(val):

    if  'shop' in val or 'store' in val or 'retail' in val or 'food' in val:
        building_mapping[val] = 'retail'
    if 'office' in val or 'commercial' in val:
        building_mapping[val] = 'commercial'
    if 'residential' in val:
        building_mapping[val] = 'apartments'
    if 'museum' in val:
        building_mapping[val] = 'museum'
    if 'ruin' in val or 'collapsed' in val:
        building_mapping[val] = 'ruins'
    if 'terrac' in val:
        building_mapping[val] = 'terrace'    
    if 'serv' in val or 'power' in val:
        building_mapping[val] = 'service'  
    if 'avia' in val:
        building_mapping[val] = 'hangar' 
    if 'mbass' in val:
        building_mapping[val] = 'embassy'
    if 'boat' in val:
        building_mapping[val] = 'houseboat'
    if 'otel' in val:
        building_mapping[val] = 'hotel'
    if 'detache' in val:
        building_mapping[val] = 'detached'  
    if  'clini' in val or 'docto' in val:
        building_mapping[val] = 'hospital'
    if  'convent' in val or 'basilica' in val or 'monastery' in val :
        building_mapping[val] = 'church'
    if  'factory' in val :
        building_mapping[val] = 'industrial'
    if  'shrine' in val :
        building_mapping[val] = 'shrine'
    if  'glass' in val :
        building_mapping[val] = 'conservatory'
    if  'enter' in val or 'hall' in val:
        building_mapping[val] = 'civic'
        
    if val in building_mapping.keys():
        return building_mapping[val]
    else: 
        building_mapping[val]=val
        return val


In [11]:
building_mapping = {'Basen': 'swimming_pool',
 'ambassadors_residence': 'embassy',
 'aviary': 'hangar',
 'basilica': 'church',
 'boat': 'houseboat',
 'bunker;museum': 'museum',
 'cheap_motel': 'hotel',
 'city_hall': 'civic',
 'classrooms': 'school',
 'clinic': 'hospital',
 'collapsed': 'ruins',
 'convent': 'church',
 'day_care': 'daycare',
 'doctors': 'hospital',
 'factory': 'industrial',
 'family_house': 'house',
 'fast_food': 'retail',
 'glasshouse': 'conservatory',
 'hall': 'civic',
 'monastery': 'church',
 'office;residential': 'apartments',
 'office;retail;residential': 'apartments',
 'offices': 'commercial',
 'power': 'service',
 'residential; retail': 'apartments',
 'residential;office': 'apartments',
 'retail;office': 'commercial',
 'ruin': 'ruins',
 'ruins; house': 'ruins',
 'semidetached_house': 'detached',
 'serv': 'service',
 'shop': 'retail',
 'store': 'retail',
 'terraces': 'terrace',
 'wayside_shrine': 'shrine'}

In [12]:
len(building_mapping)

35

In [13]:
def audit_v(tag):
    '''Audit v attribute and return value'''

    v = tag.attrib['v']
    #map building
    if tag.attrib['k']=='building':
        if v not in building_mapping.keys():
            cb = correct_building(v) #Thsi makes the process get stuck for some reason
            print('Mapped: ' + v + ' => ' + cb)
        else: cb = building_mapping[v]
            
        #cb = building_mapping[v]
        
        return cb
        
    # map phone
    elif tag.attrib['k']=='phone':
        cf = correct_phone(v)      
        print('Mapped: ' + v + ' => ' + cf)
        return cf
    return v

In [14]:
OSM_PATH = "warsaw_poland.osm"
NODES_PATH = OSM_PATH[:-4] + "_nodes.csv"
NODE_TAGS_PATH =OSM_PATH[:-4] + "_nodes_tags.csv"
WAYS_PATH = OSM_PATH[:-4] + "_ways.csv"
WAY_NODES_PATH = OSM_PATH[:-4] + "_ways_nodes.csv"
WAY_TAGS_PATH = OSM_PATH[:-4] + "_ways_tags.csv"

LOWER_COLON = re.compile(r'^([a-z]|_)+:([a-z]|_)+')
PROBLEMCHARS = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')

SCHEMA = schema.schema

# Make sure the fields order in the csvs matches the column order in the sql table schema
NODE_FIELDS = ['id', 'lat', 'lon', 'user', 'uid', 'version', 'changeset', 'timestamp']
NODE_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_FIELDS = ['id', 'user', 'uid', 'version', 'changeset', 'timestamp']
WAY_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_NODES_FIELDS = ['id', 'node_id', 'position']


def audit_k(tag, entries, default_tag_type):
    '''Audit k attribute of element and return proper k and type values'''

    k = tag.attrib['k']
    if not PROBLEMCHARS.search(k):
        index = k.find(':')
        if index == -1:
            entries['key'] = k
            entries['type']= default_tag_type
        else: 
            entries['key'] = k[index+1:]
            entries['type']= k[:index]
    else:        
        entries['type']= default_tag_type 
                    
    return entries
    


def shape_element(element, node_attr_fields=NODE_FIELDS, way_attr_fields=WAY_FIELDS,
                  problem_chars=PROBLEMCHARS, default_tag_type='regular'):
    """Clean and shape node or way XML element to Python dict"""

    node_attribs = {}
    way_attribs = {}
    way_nodes = []
    tags = []  # Handle secondary tags the same way for both node and way elements

    # YOUR CODE HERE
    if element.tag == 'node':
        
        # Iterate over node fields
        for tag in element.iter("node"):
            for field in node_attr_fields:
                node_attribs[field]= tag.attrib[field]
                
        # Iterate over tag fields        
        for tag in element.iter('tag'):
            entries={}
            entries['id']= node_attribs['id']
            
            # audit the v atttribute
            try: entries['value'] = audit_v(tag)
            except: entries['value']  = tag.attrib('v')

            
            # audit the k attribute
            entries = audit_k(tag, entries, default_tag_type)
            tags.append(entries) 
        return {'node': node_attribs, 'node_tags': tags}
    
    elif element.tag == 'way':
        
        # Iterate over way fields
        for tag in element.iter('way'):
            for field in way_attr_fields:
                way_attribs[field]= tag.attrib[field]
                
        # Iterate over tag fields        
        for tag in element.iter('tag'):
            entries={}
            entries['id']= way_attribs['id']
            
            # audit the v atttribute
            try: entries['value'] = audit_v(tag)
            except: entries['value']  = tag.attrib('v')


            
            # audit the k attribute
            entries = audit_k(tag, entries, default_tag_type)
            tags.append(entries)
            
        # Iterate over nd fields
        i=0
        for tag in element.iter('nd'):
            entriesnd={}
            entriesnd['id']= way_attribs['id']
            entriesnd['node_id'] = tag.attrib['ref']
            entriesnd['position'] = i
            i+=1
            way_nodes.append(entriesnd)    
        return {'way': way_attribs, 'way_nodes': way_nodes, 'way_tags': tags}


# ================================================== #
#               Helper Functions                     #
# ================================================== #
def get_element(osm_file, tags=('node', 'way', 'relation')):
    """Yield element if it is the right type of tag"""

    context = ET.iterparse(osm_file, events=('start', 'end'))
    _, root = next(context)
    for event, elem in context:
        if event == 'end' and elem.tag in tags:
            yield elem
            root.clear()


def validate_element(element, validator, schema=SCHEMA):
    """Raise ValidationError if element does not match schema"""
    if validator.validate(element, schema) is not True:
        field, errors = next(validator.errors.iteritems())
        message_string = "\nElement of type '{0}' has the following errors:\n{1}"
        error_string = pprint.pformat(errors)
        
        raise Exception(message_string.format(field, error_string))


class UnicodeDictWriter(csv.DictWriter, object):
    """Extend csv.DictWriter to handle Unicode input"""

    def writerow(self, row):
        super(UnicodeDictWriter, self).writerow({
            k: (v.encode('utf-8') if isinstance(v, unicode) else v) for k, v in row.iteritems()
        })

    def writerows(self, rows):
        for row in rows:
            self.writerow(row)


# ================================================== #
#               Main Function                        #
# ================================================== #
def process_map(file_in, validate):
    """Iteratively process each XML element and write to csv(s)"""

    with codecs.open(NODES_PATH, 'w') as nodes_file, \
         codecs.open(NODE_TAGS_PATH, 'w') as nodes_tags_file, \
         codecs.open(WAYS_PATH, 'w') as ways_file, \
        codecs.open(WAY_NODES_PATH, 'w') as way_nodes_file, \
         codecs.open(WAY_TAGS_PATH, 'w') as way_tags_file:

        nodes_writer = UnicodeDictWriter(nodes_file, NODE_FIELDS)
        node_tags_writer = UnicodeDictWriter(nodes_tags_file, NODE_TAGS_FIELDS)
        ways_writer = UnicodeDictWriter(ways_file, WAY_FIELDS)
        way_nodes_writer = UnicodeDictWriter(way_nodes_file, WAY_NODES_FIELDS)
        way_tags_writer = UnicodeDictWriter(way_tags_file, WAY_TAGS_FIELDS)

        nodes_writer.writeheader()
        node_tags_writer.writeheader()
        ways_writer.writeheader()
        way_nodes_writer.writeheader()
        way_tags_writer.writeheader()

        validator = cerberus.Validator()

        for element in get_element(file_in, tags=('node', 'way')):
            el = shape_element(element)
            if el:
                if validate is True:
                    validate_element(el, validator)

                if element.tag == 'node':
                    nodes_writer.writerow(el['node'])
                    node_tags_writer.writerows(el['node_tags'])
                elif element.tag == 'way':
                    ways_writer.writerow(el['way'])
                    way_nodes_writer.writerows(el['way_nodes'])
                    way_tags_writer.writerows(el['way_tags'])


if __name__ == '__main__':
    # Note: Validation is ~ 10X slower. For the project consider using a small
    # sample of the map when validating.
    
    process_map(OSM_PATH, validate=False)
    print('Processed {} Map Succesfully!'.format(OSM_PATH[:-4]))

Mapped: +48 225827500 => +48 22 5827500
Mapped: 662 350 728 => +48 66 2350728
Mapped: +48 22 833 56 60 => +48 22 8335660
Mapped: +48 22 822 27 50 => +48 22 8222750
Mapped: 22 870 03 07;22 870 03 76;517 129 105 => +48 22 8700307
Mapped: 226213280 => +48 22 6213280
Mapped: +48 22 665 975 005 => +48 22 6659750
Mapped: entrance => entrance
Mapped: +48 22 536 36 36 => +48 22 5363636
Mapped: +48 22 644 85 10 => +48 22 6448510
Mapped: +48 22 643 59 25 => +48 22 6435925
Mapped: +48 222 130 677 => +48 22 2130677
Mapped: +48 22 641 95 44 => +48 22 6419544
Mapped: +48226416331 => +48 22 6416331
Mapped: yes => yes
Mapped: +48 22 885 00 20 => +48 22 8850020
Mapped: +48 22 8410693; +48 22 8410066 => +48 22 8410693
Mapped: +48 22 644 75 25 => +48 22 6447525
Mapped: +48227570103 => +48 22 7570103
Mapped: +48228538288 => +48 22 8538288
Mapped: +48227564750 => +48 22 7564750
Mapped: 888 954 017 => +48 95 4017
Mapped: +48227163860 => +48 22 7163860
Mapped: +48 22 644 29 02 => +48 22 6442902
Mapped: +4822

In [66]:
building_mapping

{'Basen': 'swimming_pool',
 'Budynek Zabytkowy': 'Budynek Zabytkowy',
 'Lokal': 'Lokal',
 'National_Monument': 'National_Monument',
 u'Plutonowego J\xf3zefa Cie\u0107wierza': u'Plutonowego J\xf3zefa Cie\u0107wierza',
 'ambassadors_residence': 'embassy',
 'apartments': 'apartments',
 'apartments;commercial': 'commercial',
 'aviary': 'hangar',
 'barn': 'barn',
 'basilica': 'church',
 'belfry': 'belfry',
 'boat': 'houseboat',
 'boathouse': 'houseboat',
 'bridge': 'bridge',
 'bunker': 'bunker',
 'bunker;museum': 'museum',
 'cabin': 'cabin',
 'cage': 'cage',
 'car_shop': 'retail',
 'castle': 'castle',
 'cathedral': 'cathedral',
 'chapel': 'chapel',
 'cheap_motel': 'hotel',
 'church': 'church',
 'city_hall': 'civic',
 'civ': 'civ',
 'civic': 'civic',
 'classrooms': 'school',
 'clinic': 'hospital',
 'collapsed': 'ruins',
 'commercial': 'commercial',
 'construction': 'construction',
 'container': 'container',
 'convent': 'church',
 'corridor': 'corridor',
 'cowshed': 'cowshed',
 'day_care': 'd